<a href="https://colab.research.google.com/github/suzzini/DS_Project/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_0501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- train_test_split
- exclude
- select_dtypes
- nunique
- [for]

In [1]:
import pandas as pd

target = 'vacc_h1n1_f' #독감
# target = 'vacc_seas_f' #계절독감
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

### 데이터 나누기**(train_test_split)**

In [2]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)
#train, test의 비율 8/2
#훈련데이터를 나눌 때 무작위로 샘플링을 하되, original dataset의 클래스 비율이 train, test set에서도 동일하게 유지되는 것을 보장한다

train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

#### 타켓의 비율 확인하기

In [5]:
print(train[target].value_counts())

train[target].value_counts(normalize=True) # 백분율로 보여줌 

0    25661
1     8062
Name: vacc_h1n1_f, dtype: int64


0    0.760935
1    0.239065
Name: vacc_h1n1_f, dtype: float64

#### exclude를 통해 데이터 타입이 number가 아닌 특성들 확인하기

In [13]:
train.describe(exclude='number')

,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,agegrp,employment_status,census_msa,employment_industry,employment_occupation,state
count,26637,26602,26568,26519,26466,26413,33723,25466,33723,13714,13714,33723
unique,6,6,6,6,6,6,7,3,3,23,25,51
top,Somewhat Effective,Somewhat Low,Not Very Worried,Somewhat Effective,Somewhat Low,Not At All Worried,65+ Years,Employed,"MSA, Not Principle City",Health Care and Social Assistance Industries,Management Occupations,CALIFORNIA
freq,11710,9980,9229,11571,9057,11813,6821,13722,15202,2451,1769,829


In [15]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
employment_status,25466,3,Employed,13722
census_msa,33723,3,"MSA, Not Principle City",15202
opinion_h1n1_vacc_effective,26637,6,Somewhat Effective,11710
opinion_h1n1_risk,26602,6,Somewhat Low,9980
opinion_h1n1_sick_from_vacc,26568,6,Not Very Worried,9229
opinion_seas_vacc_effective,26519,6,Somewhat Effective,11571
opinion_seas_risk,26466,6,Somewhat Low,9057
opinion_seas_sick_from_vacc,26413,6,Not At All Worried,11813
agegrp,33723,7,65+ Years,6821
employment_industry,13714,23,Health Care and Social Assistance Industries,2451


In [18]:
train.shape

(33723, 39)

In [22]:
#원하는 데이터 타입가져오기
selected_cols = train.select_dtypes(include=['number', 'object'])
selected_cols

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,agegrp,education_comp,raceeth4_i,sex_i,inc_pov,marital,rent_own_r,employment_status,census_region,census_msa,n_adult_r,household_children,n_people_r,employment_industry,employment_occupation,hhs_region,state,vacc_h1n1_f
20365,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,Somewhat Effective,Somewhat Low,Not At All Worried,Very Effective,Somewhat Low,Not At All Worried,18 - 34 Years,4.0,3,1,1,2.0,1.0,Employed,1,"MSA, Not Principle City",4.0,0.0,4.0,"Professional, Scientific, and Technical Servic...",Office and Administrative Support Occupations,2,NEW JERSEY,0
29138,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,Somewhat Effective,Somewhat High,Not Very Worried,Somewhat Effective,Somewhat Low,Not Very Worried,35 - 44 Years,2.0,3,2,3,2.0,2.0,Unemployed,2,Non-MSA,2.0,2.0,4.0,NaN,NaN,7,IOWA,1
36128,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,Not Very Effective,Very Low,Dont Know,Somewhat Effective,Very Low,Very Worried,65+ Years,1.0,3,2,2,2.0,1.0,Not in Labor Force,4,"MSA, Not Principle City",1.0,0.0,1.0,NaN,NaN,10,OREGON,0
39793,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Somewhat Low,Not Very Worried,Somewhat Effective,Very Low,Not At All Worried,55 - 64 Years,4.0,3,1,1,1.0,1.0,Employed,2,"MSA, Principle City",2.0,0.0,2.0,"Arts, Entertainment, and Recreation Industries...",Business and Financial Operations Occupations,7,IOWA,0
32973,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Not At All Effective,Somewhat Low,Not At All Worried,Not At All Effective,Very Low,Not At All Worried,18 - 34 Years,4.0,3,2,2,1.0,1.0,Not in Labor Force,2,Non-MSA,2.0,3.0,5.0,NaN,NaN,7,IOWA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 Months - 9 Years,NaN,3,1,4,NaN,1.0,NaN,2,"MSA, Not Principle City",4.0,1.0,5.0,NaN,NaN,5,ILLINOIS,0
4412,3.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,0.0,Not Very Effective,Somewhat High,Very Worried,Not Very Effective,Very High,Very Worried,55 - 64 Years,2.0,3,2,2,1.0,1.0,Not in Labor Force,2,"MSA, Not Principle City",2.0,0.0,2.0,NaN,NaN,5,INDIANA,0
19064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 Months - 9 Years,NaN,3,1,2,NaN,1.0,NaN,3,Non-MSA,2.0,1.0,3.0,NaN,NaN,4,GEORGIA,0
40906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 - 17 Years,NaN,3,1,1,NaN,1.0,NaN,4,Non-MSA,3.0,1.0,4.0,NaN,NaN,8,MONTANA,0


In [25]:
labels =selected_cols.nunique() # 특성별 카디널리티 리스트
labels[labels <= 30].index.tolist() #특성의 값이 30개 이하인 컬럼명만 가져오기

['h1n1_concern',
 'h1n1_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_insurance',
 'health_worker',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'agegrp',
 'education_comp',
 'raceeth4_i',
 'sex_i',
 'inc_pov',
 'marital',
 'rent_own_r',
 'employment_status',
 'census_region',
 'census_msa',
 'n_adult_r',
 'household_children',
 'n_people_r',
 'employment_industry',
 'employment_occupation',
 'hhs_region',
 'vacc_h1n1_f']

In [26]:
[col for col in train.columns if 'behavioral' in col]

['behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face']

In [27]:
[col for col in train.columns if ('employment' in col or 'seas' in col)]

['doctor_recc_seasonal',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'employment_status',
 'employment_industry',
 'employment_occupation']